# 第三章 推薦システムの高速化

## モデルベースの推薦システム

第二章の手法は**メモリベース**と呼ばれる方法でした。今回は**モデルベース**を試します。

## 分類モデルを用いた内容ベースフィルタリング

さっきと一緒じゃね？
分類モデルとは**すでにラベル付されたデータを元に統計的な手法を用いて、ラベル付いていないデータに特定のラベルを与える手法です。**

分類モデルの種類には

* ナイーブベイズ（NB）
* ロジスティック回帰（LR）
* サポートベクターマシーン（SVM）

今回はナイーブベイズを使います。
本にはナイーブベイズは簡単に組めるけどLRとSVMに対し、制度が劣るらしいです。

そんなデメリットに対しメリットとして

* モデルをつくるための計算時間が短い
* 新しいデータに対してモデルの更新がしやすい
* モデルの中身が見やすく、手動でのチューニングが容易

ナイーブベイズの中身についての説明

$P(c|X) = \dfrac{P(c,X)}{P(X)} = \dfrac{P(X|c)P(c)}{P(X)}$

ある事象Xが発生したとき、それが暮らすcに属する確率$P(c|X)$を求めることでクラス分類を行います。

もっとよく知りたい方はこちら

* X=ガンダム、Y={連邦軍, ジオン軍（連邦軍じゃない）}
* X=RX78-2、Y={ガンダム,ザク,ジオング}
* X=赤いMS、Y={シャアザク,ガンタンク,ズゴッグ}

のように、Xには何らかの起こった事象や入力データを、Yにはそこから推論したい事柄などを当てはめるわけです。
※尤度 ≒ 条件付き確率

http://qiita.com/aflc/items/13fe52243c35d3b678b0
※最初はわかりやすいが後半数式ばっかで涙目

では、事象Xがn次元空間上のベクトル表現として$X=(x_1,x_2,\dots,x_n)$と表せるときに、特徴空間上で独立であると仮定すれば、$P(X|c)$は次のように表せます。

$P(X|c) = P(x_1,x_2,\dots,x_n|c) = P(x_1|c)P(x_2|c) \cdots P(x_n|c)$

ここである事象Xに対して、もっとも$P(c|X)$が大きくなるクラスcを与える問題を考えれば$P(X)$は共通になります。

$P(c)(x_1|c)(x_2|c) \cdots P(x_n|c)$

が、最大になるcを与えればいいということがわかります（わかります？）

このとき一般に$P(x_i|c)$は微小になりアンダフローを起こしやすいため、次のように対数表現で表します。

$log P(c) + log P(x_1|c) + log P (x_2|c) + \cdots + log P(x_n|c)$

このとき最大化するクラスcを求めます。これがナイーブベイズ分類器です。このナイーブベイズ分類器を用いた文章推薦システムを考えます。

今回はユーザーにニュースなどの文章を推薦するシステムで、ユーザーはその中で読みたくなかった文章についてはシステムにフィードバッグできるしくみがあるとします。ユーザーが読んだ文章のクラスを正解クラス、ユーザーがよみたくなかった文章のクラスを不正解クラスとし、ナイーブベイズ分類器に酔って正解クラスに分類されるものが推薦される文章です。

まず文章をベクトル表現します。文章を単語の集合として表現することを考えます。この表現方法を**Bag of Words**と呼びます。


ある文章dが単語$w_1,\cdots,w_N$で表現されるとき、確率は次のように計算できます。

$P(w_i|c) = \dfrac{クラスの文章で単語w_iを含む文章数}{クラスの文章数}$


ナイーブベイズはscikit-learnでできるんだって！すげぇ！

In [1]:
from sklearn import datasets
iris = datasets.load_iris() # サンプルデータセットの"iris（花？）"を取得
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB() # ガウシアンなイーブイズ分類器をセット
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data) # 入力データ iris.data、出力データ iris.targetとして学習（fit）してテストデータを投げる（predict)

print(y_pred) # んで、結果とね。

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [2]:
iris.data.shape[0] 

150

In [3]:
(iris.target != y_pred).sum() # 予測したデータ（学習データ）と入力データを比較したら6個間違いがある

6

## 次元削減を用いた協調フィルタリング



In [7]:
import numpy
import top_modeling as tm

In [11]:
R = numpy.array([
  [5, 3, 0, 1],
  [4, 0, 0, 1],
  [1, 1, 0, 5],
  [1, 0, 0, 4],
  [0, 1, 5, 4]
])

nP, nQ = tm.matrix_factorization(R, 2)
nR = numpy.dot(nP.T, nQ)

print(nR)

[[ 5.0192881   2.9479985   4.44135127  0.99521866]
 [ 3.98040699  2.34580851  3.70550433  0.99497778]
 [ 1.07584801  0.81719661  5.21117303  4.99113573]
 [ 0.96223933  0.71246821  4.23721761  3.98882969]
 [ 1.84884681  1.22872866  4.91890374  4.04926914]]
